In [ ]:
from torch import nn

from models.AudioResnet import AudioResnet


class TinyCNN(nn.Module):
    def __init__(self, num_classes=50):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((12, 32)),
            nn.Flatten(),
            nn.Linear(32 * 12 * 32, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
from data.procesing import ParseData
directory = "tiny-melspec-dataset-10"

ParseData("raw_30s_cleantags_125artists", "E:/SongsDataset/raw_30s_melspecs/", f"E:/SongsDataset/{directory}", features=96, chunks_per_batch=4096, chunk_size=256, songs_per_label=1200, labels_to_include=10, chunks_per_song=16, test_prob=0.1, convert=False)

In [ ]:
from data.data import *

augmentations = Compose([
    AddGaussianNoise(std=0.05),
     TimeMasking(max_mask_pct=0.075),
     FrequencyMasking(max_mask_pct=0.075),
])

class Config:
    # === General ===
    model_name = "TinyCNN-10"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.float32
    save_path = f"trained_models\\{model_name}\\"
    seed = 42

    # === Training ===
    num_classes = 10
    num_epochs = 20
    batch_size = 64
    learning_rate = 3e-5
    weight_decay = 1e-5
    warmup_percent = 1.0 / 30.0
    save_checkpoints = True

    # === Dataset ===
    transforms = None
    use_masks = True
    num_workers = None
    prefetch_factor = num_workers
    val_split = 0.1
    shuffle = True
    pos_weights = (torch.ones(num_classes) * num_classes)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights.to("cuda"))


In [ ]:
from torch.utils.data import DataLoader

directory = "tiny-melspec-dataset-10"
large_directory = "large-melspec-dataset-50-MTG"

tiny_train_dataset = AudioDataset(f"E:\\SongsDataset\\{large_directory}\\train_set\\data", f"E:\\SongsDataset\\{large_directory}\\train_set\\genre_labels", augmentations)
tiny_test_dataset = AudioDataset(f"E:\\SongsDataset\\{large_directory}\\test_set\\data", f"E:\\SongsDataset\\{large_directory}\\test_set\\genre_labels", augmentations)

train_dataloader = DataLoader(
    tiny_train_dataset,
    batch_size=Config.batch_size,
    shuffle=True,
    num_workers=Config.num_workers,
    prefetch_factor=Config.prefetch_factor,
)

test_dataloader = DataLoader(
    tiny_test_dataset,
    batch_size=Config.batch_size,
    shuffle=True,
    num_workers=Config.num_workers,
    prefetch_factor=Config.prefetch_factor,
)

In [ ]:
from training.classification_training import train_classifier
from models.ShortChunkCNN import ShortChunkCNN
from utils import misc

model = TinyCNN(num_classes=10)
print(f"{misc.model_size(model)} Parameters")

train_classifier(model, test_dataloader, train_dataloader, Config)

In [ ]:
import matplotlib.pyplot as plt

def show_mel(mel, label):
    plt.imshow(mel.squeeze(), origin='lower', aspect='auto', cmap='magma')
    plt.title(f"Genres: {torch.nonzero(label).squeeze().tolist()}")
    plt.colorbar()
    plt.show()

train_dataloader[0]

In [ ]:
#from loss.loss_utils import DifferentiablePRAUCLoss

pos_weights = (torch.ones(10) * 10).to("cuda")
#model = torch.load("E:/Coding/SongAnalyzer/Analyzer/src/trained_models/ShortChunkCNN/Classifier-Epoch-55.pt", weights_only=False)

criterion = (nn.BCEWithLogitsLoss(pos_weight=pos_weights))# + DifferentiablePRAUCLoss(tau=0.05))
test_loss_average, all_probs, all_labels = eval(model, test_dataloader, criterion, Config)

all_p_tensor = torch.stack([torch.tensor(x) for x in all_probs], dim=0).float()
all_l_tensor = torch.stack([torch.tensor(x) for x in all_labels], dim=0).int()

In [ ]:
from sklearn.metrics import average_precision_score

for i in range(10):
    precision, recall, thresholds = precision_recall_curve(all_l_tensor[:, i], all_p_tensor[:, i])
    ap = average_precision_score(all_l_tensor[:, i], all_p_tensor[:, i])
    plt.plot(recall, precision, label=f"Class {i} (AP={ap:.2f})")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.title("Per-Class Precision-Recall Curves")
plt.grid(True)
plt.show()

ap_per_class = [average_precision_score(all_l_tensor[:, i], all_p_tensor[:, i]) for i in range(10)]
print("AP per class:", ap_per_class)